# Базы данных

В современном мире больших данных все еще большая доля приходится на классические реляционные базы данных, которые представляют из себя совокупность таблиц, объединенных по внешним ключам. Именно такой тип используется в работе веб-страниц. Наиболее популярным примером является база данных [PostgreSQL](https://postgrespro.ru/docs/postgresql/14/index) (сокращенно - postgres). Наиболее распространенными также являются MySQL, Oracle, MariaDB, SQLite.

Про остальные типы данных и баз данных можно почитать [здесь](https://proglib.io/p/11-tipov-sovremennyh-baz-dannyh-kratkie-opisaniya-shemy-i-primery-bd-2020-01-07)

В этом курсе мы будем говорить про команды в postgres, однако существенных различий с другими языками SQL-запросов нет.
Хороший самоучитель по postgres находится [здесь](https://www.pgexercises.com/gettingstarted.html). Рекомендуется изучать документацию с самого начала и начинать тренировать простейшие упражнения, где в конце объясняются, как работает запрос.



# Работа с базой данных в Python

В Python мы работаем с базой данных postgres через модуль *psycopg2*. Однако, чтобы подключиться к существующей, мы должны знать:
- имя базы
- имя пользователя
- пароль
- хост (либо локальный, либо через http)
- номер порта (для локального хоста - это, как правило, 5432)

Ниже представлен код создания нашей базы на локальном хосте при помощи модуля *sqllite3* (потому что для создания базы postgres нам нужно предварительно установить его на компьютер, а это нам сейчас не нужно).



In [6]:
import psycopg2
import sqlite3


def create_connection():
    """ create a database connection to a database that resides
        in the memory
    """
    conn = None
    try:
        conn = sqlite3.connect(':memory:')
        print(sqlite3.version)
    except sqlite3.Error as e:
        print(e)
    return conn
conn = create_connection()

2.6.0


Теперь создадим таблицы [отсюда](https://www.pgexercises.com/gettingstarted.html)

In [28]:
c = conn.cursor()
c.execute("""
    CREATE TABLE members
    (
       memid integer NOT NULL,
       surname character varying(200) NOT NULL,
       firstname character varying(200) NOT NULL,
       address character varying(300) NOT NULL,
       zipcode integer NOT NULL,
       telephone character varying(20) NOT NULL,
       recommendedby integer,
       joindate timestamp NOT NULL,
       CONSTRAINT members_pk PRIMARY KEY (memid)
       CONSTRAINT fk_members_recommendedby FOREIGN KEY (recommendedby)
       REFERENCES members(memid) ON DELETE SET NULL
    );
""")

c.execute("""CREATE TABLE facilities
    (
       facid integer NOT NULL,
       name character varying(100) NOT NULL,
       membercost numeric NOT NULL,
       guestcost numeric NOT NULL,
       initialoutlay numeric NOT NULL,
       monthlymaintenance numeric NOT NULL,
       CONSTRAINT facilities_pk PRIMARY KEY (facid)
    );
    """)

c.execute("""
        CREATE TABLE bookings
        (
       bookid integer NOT NULL,
       facid integer NOT NULL,
       memid integer NOT NULL,
       starttime timestamp NOT NULL,
       slots integer NOT NULL,
       CONSTRAINT bookings_pk PRIMARY KEY (bookid),
       CONSTRAINT fk_bookings_facid FOREIGN KEY (facid) REFERENCES facilities(facid),
       CONSTRAINT fk_bookings_memid FOREIGN KEY (memid) REFERENCES members(memid)
       );
    """)

Теперь посмотрим на список доступных таблиц в нашей базе, которая хранится у нас в оперативной памяти.

In [43]:
exec = c.execute(
    """
SELECT
    name
FROM
    sqlite_schema
WHERE
    type ='table'
    """)
result = c.fetchall()
result

[('members',), ('facilities',), ('bookings',)]

Отметим, что синтаксис SQLite может немного отличаться от PostgreSQL. Однако, для тренировки базовых запросов это не имеет значения.

# Анализ базы данных в pandas

pandas - это огромная библиотека для анализа данных. Это базовый инструмент для работы с анализом данных в Python.
Однако, разработчикам необязательно знать данный пакет углубленно. Достаточно разбираться в базовых функциям, чтобы было удобно анализировать данные, приходящие из баз данных. Ниже представлен пример импорта данных из таблицы *members*.

In [51]:
import pandas as pd
query = \
"""
SELECT *
FROM members
"""
results_df = pd.read_sql_query(query, conn)
results_df

,memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate


Видим, что у нас нет никаких данных, кроме названий столбцов. Это логично, ведь при создании таблицы мы указали лишь название и тип данных. При помощи метода ```pd.read_sql_query``` мы можем выгружать данные в удобном формате. Однако, изменять данные мы будем чуть более сложным способом через ```conn().cursor.execute()```. Добавим также, что производить трудоемкие для оперативной памяти операции через *pandas* не рекомендуется, поскольку от неэффективно работает с памятью. Рекомендуется большую часть обработки данных производить через SQL-запросы, а через pandas выводить основной результат.

In [49]:
c.execute(
"""
INSERT INTO facilities
VALUES
(0, 'Tennis Court 1', 5, 25, 10000, 200),
(1, 'Tennis Court 2',	5,	25,	8000,	200);
""")

In [50]:
query = \
"""
SELECT *
FROM facilities
"""
results_df = pd.read_sql_query(query, conn)
results_df

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5,25,10000,200
1,1,Tennis Court 2,5,25,8000,200


Теперь у нас появились данные! Давайте добавим еще больше строк ко всем 3 таблицам.

In [56]:
c.execute(
"""
INSERT INTO facilities
VALUES
(2, 'Badminton Court', 0,	15.5,	4000,	50),
(3,	'Table Tennis', 0,	5,	320,	10),
(4, 'Massage Room 1',	35,	80,	4000,	3000),
(5, 'Massage Room 2',	35,	80,	4000,	3000),
(6, 'Squash Court', 3.5,	17.5,	5000,	80),
(7, 'Snooker Table', 0, 5, 450, 15),
(8,	'Pool Table',	0,	5,	400,	15);
""")

c.execute(
"""
INSERT INTO members
VALUES
(4,	'Joplette', 'Janice',	'20 Crossing Road, New York',	234,	'(833) 942-4710',	1,	'2012-07-03 10:25:05'),
(5, 'Butters',	'Gerald',	'1065 Huntingdon Avenue, Boston',	56754,	'(844) 078-4130',	1,	'2012-07-09 10:44:09');
"""
)

c.execute(
"""
INSERT INTO bookings
VALUES
(0,	3,	1,	'2012-07-03 11:00:00',	2),
(1,	4,	1,	'2012-07-03 08:00:00',	2),
(2,	6,	0,	'2012-07-03 18:00:00',	2),
(3,	7,	1,	'2012-07-03 19:00:00',	2)
"""
)

# Немного графического анализа